<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

In [1]:
# # This cell is for papermill parameters. DO NOT CHANGE THE VARIABLE NAMES.
# # Default values for manual execution (papermill will override these)
# input_geojson_path = None
# output_csv_path = None

## Step 1.1: Configure the environment

In [2]:
import os
from datetime import datetime
from shapely.geometry import Polygon
from shapely import box
from pyproj import CRS 
import folium
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray
from ipyleaflet import basemaps
from numpy.lib.stride_tricks import sliding_window_view
import pystac_client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import planetary_computer
# from odc.stac import load
from pystac.client import Client
from skimage.feature import graycomatrix, graycoprops
from utils import load_data, load_s1_dem, scale, calculate_band_indices, apply_mask, mask_water, all_masks, do_prediction

In [3]:
# Reload scripts and imports
%load_ext autoreload
%autoreload 2

In [4]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Enter your site name
site = "tongatapu"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}_postcard_4"
print(version)

nm-tongatapu-11122025_postcard_4


In [5]:
gdfs = []
postcards_path = "training-data/"
file_extension: str = ".geojson"

for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
    # try:
        gdf = gpd.read_file(file_path)
        gdfs.append(gdf)

In [6]:
for filename in os.listdir(postcards_path):
    file_path = os.path.join(postcards_path, filename)
    if os.path.isfile(file_path) and filename.endswith(file_extension):
        print(filename) # This line will print the name of each GeoJSON file
        # The rest of your code to read the file and append to gdfs
        # gdf = gpd.read_file(file_path)
        # gdfs.append(gdf)

print("\nFinished listing GeoJSON files.")

nm-tongatapu-11122025_postcard_4.geojson

Finished listing GeoJSON files.


## Step 1.2: Configure STAC access and search parameters

In [7]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

In [8]:
training = gpd.read_file(f"training-data/{version}.geojson")
training = training.to_crs("EPSG:4326")
min_lon, min_lat, max_lon, max_lat = training.total_bounds

bbox = [min_lon, min_lat, max_lon, max_lat]

In [9]:
training

,LULC_code,LULC_class,geometry
0,6,Water,POINT (-175.32648 -21.05595)
1,6,Water,POINT (-175.36156 -21.09345)
2,6,Water,POINT (-175.12939 -21.27945)
3,6,Water,POINT (-175.02447 -21.1477)
4,6,Water,POINT (-175.31365 -21.16867)
...,...,...,...
985,4,Settlements,POINT (-175.1994 -21.13518)
986,4,Settlements,POINT (-175.19913 -21.13671)
987,4,Settlements,POINT (-175.20174 -21.13612)
988,4,Settlements,POINT (-175.20039 -21.13224)


In [10]:
print(training['LULC_class'].value_counts())
print('total gps points',(len(training)))

LULC_class
Grazing_Cropland    310
Forest_land         200
Settlements         140
Wetland             120
Water               110
Bare_land           110
Name: count, dtype: int64
total gps points 990


In [11]:
datetime = "2023"

items = client.search(
    collections=["dep_s2_geomad"],
    datetime=datetime,
    bbox=bbox
).item_collection()

print(f"Found {len(items)} items in for {datetime}")

Found 1 items in for 2023


In [12]:
measurements = ["nir", "red", "blue", "green", "emad", "smad", "bcmad", "green", "nir08", "nir09", "swir16", "swir22", "coastal", "rededge1", "rededge2", "rededge3"]
data = load_data(
    items,
    measurements,
    bbox,
)
    
# Now you can use the 'data' variable
print(data)

<xarray.Dataset> Size: 359MB
Dimensions:      (y: 2654, x: 3753, time: 1)
Coordinates:
  * y            (y) float64 21kB -2.383e+06 -2.383e+06 ... -2.41e+06 -2.41e+06
  * x            (x) float64 30kB 3.856e+06 3.856e+06 ... 3.893e+06 3.893e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2023-01-01
Data variables: (12/15)
    nir          (time, y, x) uint16 20MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    red          (time, y, x) uint16 20MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    blue         (time, y, x) uint16 20MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) uint16 20MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    emad         (time, y, x) float32 40MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    smad         (time, y, x) float32 40MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    ...           ...
    swir16       (time, y, x) uint16 20MB

In [13]:
dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
configure_s3_access(cloud_defaults=True, requester_pays=True)

In [14]:
scaled = scale(data)
scaled = scaled.compute().squeeze()

In [15]:
# Explore the site we are working on
# scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [16]:
scaled

<xarray.Dataset> Size: 1GB
Dimensions:      (y: 2654, x: 3753)
Coordinates:
  * y            (y) float64 21kB -2.383e+06 -2.383e+06 ... -2.41e+06 -2.41e+06
  * x            (x) float64 30kB 3.856e+06 3.856e+06 ... 3.893e+06 3.893e+06
    spatial_ref  int32 4B 3832
    time         datetime64[ns] 8B 2023-01-01
Data variables: (12/15)
    nir          (y, x) float64 80MB 0.0423 0.0435 0.0455 ... 0.0384 0.0389
    red          (y, x) float64 80MB 0.0462 0.0461 0.0494 ... 0.0409 0.0434
    blue         (y, x) float64 80MB 0.0625 0.0622 0.0646 ... 0.0607 0.0602
    green        (y, x) float64 80MB 0.05 0.05 0.0524 ... 0.0454 0.046 0.047
    emad         (y, x) float32 40MB 0.1268 0.1316 0.1361 ... 0.1186 0.1188
    smad         (y, x) float32 40MB 1.771e-06 1.88e-06 ... 4.219e-06 4.147e-06
    ...           ...
    swir16       (y, x) float64 80MB 0.0425 0.045 0.0467 ... 0.0397 0.0395
    swir22       (y, x) float64 80MB 0.0415 0.0448 0.0466 ... 0.0376 0.0373
    coastal      (y, x) float64 80MB 0.0716 0.0717 0.0732 ... 0.0732 0.073
    rededge1     (y, x) float64 80MB 0.048 0.0497 0.0513 ... 0.0438 0.0436
    rededge2     (y, x) float64 80MB 0.0459 0.0475 0.0492 ... 0.0421 0.0418
    rededge3     (y, x) float64 80MB 0.0472 0.0492 0.0509 ... 0.0421 0.0419

In [ ]:
dem_data = load_s1_dem("cop-dem-glo-30", None, bbox)

# Assuming 'scaled' has a valid CRS (based on the merged output)
print("Scaled CRS:", scaled.rio.crs)
print("DEM Data CRS:", dem_data.rio.crs)
# --- Inspection Block ---

print("--- Scaled Data ---")
print("Dims:", scaled.dims)
print("x values:", scaled.x.values)
print("y values:", scaled.y.values)
print("CRS:", scaled.rio.crs)

print("\n--- DEM Data ---")
print("Dims:", dem_data.dims)
# NOTE: Replace 'x' and 'y' below with the actual coordinate names if they are different (e.g., 'lon', 'lat')
print("x values:", dem_data.lon.values if 'lon' in dem_data.coords else dem_data.x.values)
print("y values:", dem_data.lat.values if 'lat' in dem_data.coords else dem_data.y.values)
print("CRS:", dem_data.rio.crs)

import xarray as xr
import rioxarray
from rasterio.enums import Resampling 
import rasterio # Needed for Resampling

# Prepare DEM (rename band)
# dem_data is a Dataset with a 'data' variable.
dem_data = dem_data.rename({"data": "dem"})

# --- Get CRS Information ---
target_crs = scaled.rio.crs
target_resolution = scaled.rio.resolution()
resampling_method = Resampling.nearest # Passed as the required enum

# --- REPROJECTION (From EPSG:4326 to EPSG:3832) ---
# This is the CRITICAL step that converts degree coordinates to meter coordinates
# and resamples them onto the correct grid space.

# Reproject DEM data
dem_reprojected = dem_data.rio.reproject(
    dst_crs=target_crs,
    resampling=resampling_method,
    resolution=target_resolution
)
# --- FINAL ALIGNMENT & MERGE ---

# snaps the reprojected data to the exact y/x coordinates of 'scaled'.
dem_final = dem_reprojected.reindex_like(scaled, method='nearest')

# Combine final aligned datasets
data = xr.merge([scaled, dem_final], compat='override')

print("Merge successful. Inspecting final Data:")
print(data)

In [17]:
scaled = calculate_band_indices(scaled)
Dataset = scaled

In [18]:
scaled, mask = all_masks(scaled, return_mask = True)
# mask.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

In [19]:
dem_data = load_s1_dem("cop-dem-glo-30", None, bbox)

In [20]:
# Assuming 'scaled' has a valid CRS (based on the merged output)
print("Scaled CRS:", scaled.rio.crs)
print("DEM Data CRS:", dem_data.rio.crs)

Scaled CRS: EPSG:3832
DEM Data CRS: EPSG:4326


In [21]:
# --- Inspection Block ---

print("--- Scaled Data ---")
print("Dims:", scaled.dims)
print("x values:", scaled.x.values)
print("y values:", scaled.y.values)
print("CRS:", scaled.rio.crs)

print("\n--- DEM Data ---")
print("Dims:", dem_data.dims)
# NOTE: Replace 'x' and 'y' below with the actual coordinate names if they are different (e.g., 'lon', 'lat')
print("x values:", dem_data.lon.values if 'lon' in dem_data.coords else dem_data.x.values)
print("y values:", dem_data.lat.values if 'lat' in dem_data.coords else dem_data.y.values)
print("CRS:", dem_data.rio.crs)

import xarray as xr
import rioxarray
from rasterio.enums import Resampling 
import rasterio # Needed for Resampling

# Prepare DEM (rename band)
# dem_data is a Dataset with a 'data' variable.
dem_data = dem_data.rename({"data": "dem"})

# --- Get CRS Information ---
target_crs = scaled.rio.crs
target_resolution = scaled.rio.resolution()
resampling_method = Resampling.nearest # Passed as the required enum

# --- REPROJECTION (From EPSG:4326 to EPSG:3832) ---
# This is the CRITICAL step that converts degree coordinates to meter coordinates
# and resamples them onto the correct grid space.

# Reproject DEM data
dem_reprojected = dem_data.rio.reproject(
    dst_crs=target_crs,
    resampling=resampling_method,
    resolution=target_resolution
)
# --- FINAL ALIGNMENT & MERGE ---

# snaps the reprojected data to the exact y/x coordinates of 'scaled'.
dem_final = dem_reprojected.reindex_like(scaled, method='nearest')

# Combine final aligned datasets
data = xr.merge([scaled, dem_final], compat='override')

print("Merge successful. Inspecting final Data:")
print(data)

--- Scaled Data ---
Dims: FrozenMappingWarningOnValuesAccess({'y': 2654, 'x': 3753})
x values: [3855935. 3855945. 3855955. ... 3893435. 3893445. 3893455.]
y values: [-2383205. -2383215. -2383225. ... -2409715. -2409725. -2409735.]
CRS: EPSG:3832

--- DEM Data ---
Dims: FrozenMappingWarningOnValuesAccess({'y': 2236, 'x': 3372, 'time': 1})
x values: [-175.36155 -175.36145 -175.36135 ... -175.02465 -175.02455 -175.02445]
y values: [-21.05595 -21.05605 -21.05615 ... -21.27925 -21.27935 -21.27945]
CRS: EPSG:4326


In [29]:
# # Merge all datasets into one multiband dataset

# # Remove the 'time' dimension to simplify merging
# # data = scaled.squeeze("time", drop=True)
# dem_data = dem_data.squeeze("time", drop=True)
# # radar = radar.squeeze("time", drop=True)

# # Combine spectral, radar, and elevation data
# final_data = xr.merge([data, dem_data], compat='override')

# # # Rename elevation band for clarity
# # final_data = final_data.rename({
# #     "data": "dem"  # Rename 'data' to 'dem' for elevation
# # })


In [32]:
final_data

<xarray.Dataset> Size: 9GB
Dimensions:        (y: 4890, x: 7125, time: 1)
Coordinates:
  * y              (y) float64 39kB -2.41e+06 -2.41e+06 ... -21.06 -21.06
  * x              (x) float64 57kB -175.4 -175.4 -175.4 ... 3.893e+06 3.893e+06
  * time           (time) datetime64[ns] 8B 2021-04-22
    spatial_ref    int32 4B 3832
Data variables: (12/33)
    nir            (y, x) float64 279MB nan nan nan nan nan ... nan nan nan nan
    red            (y, x) float64 279MB nan nan nan nan nan ... nan nan nan nan
    blue           (y, x) float64 279MB nan nan nan nan nan ... nan nan nan nan
    green          (y, x) float64 279MB nan nan nan nan nan ... nan nan nan nan
    emad           (y, x) float32 139MB nan nan nan nan nan ... nan nan nan nan
    smad           (y, x) float32 139MB nan nan nan nan nan ... nan nan nan nan
    ...             ...
    nbi            (y, x) float64 279MB nan nan nan nan nan ... nan nan nan nan
    ndmi           (y, x) float64 279MB nan nan nan nan nan ... nan nan nan nan
    bsi            (y, x) float64 279MB nan nan nan nan nan ... nan nan nan nan
    awei           (y, x) float64 279MB nan nan nan nan nan ... nan nan nan nan
    tc_wetness     (y, x) float64 279MB nan nan nan nan nan ... nan nan nan nan
    dem            (time, y, x) float32 139MB nan nan nan nan ... nan nan nan

In [33]:
# scaled.odc.explore(vmin=0, vmax=0.3, bands=["red", "green", "blue"], crs="EPSG:3832", name=site)

### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [34]:
# Reproject training data to the GeoMAD CRS and convert to xarray
training_reprojected = training.to_crs(scaled.odc.crs)
training_da = training_reprojected.assign(
    x=training_reprojected.geometry.x, y=training_reprojected.geometry.y
).to_xarray()

# Extract training values from the masked dataset
training_values = (
    data.sel(training_da[["x", "y"]], method="nearest")
    .squeeze()
    .compute()
    .to_pandas()
)
training_values

,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,swir16,...,nbi,ndmi,bsi,awei,tc_wetness,y,x,spatial_ref,time,dem
index,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-2383205.0,3859835.0,3832,2021-04-22,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-2387655.0,3855935.0,3832,2021-04-22,0.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-2409735.0,3881775.0,3832,2021-04-22,0.000000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-2394095.0,3893455.0,3832,2021-04-22,0.000000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-2396575.0,3861265.0,3832,2021-04-22,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,0.2348,0.2107,0.1506,0.1891,0.047303,8.195043e-08,0.000003,0.2388,0.2563,0.3130,...,0.142753,-0.142753,0.148946,-0.120950,-0.053969,-2392605.0,3873985.0,3832,2021-04-22,4.704696
986,0.3164,0.2806,0.2096,0.2473,0.079557,5.739927e-08,0.000003,0.3615,0.2947,0.3866,...,0.099858,-0.099858,0.086250,-0.140475,-0.002637,-2392785.0,3874015.0,3832,2021-04-22,3.618409
987,0.2451,0.2180,0.1563,0.1883,0.046965,1.190364e-07,0.000003,0.2389,0.2742,0.3012,...,0.102691,-0.102691,0.119061,-0.119425,-0.037975,-2392715.0,3873725.0,3832,2021-04-22,8.095431


In [35]:
training_values.columns

Index(['nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad', 'nir08',
       'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'swir22_swir16', 'mci', 'ndci', 'nbi', 'ndmi', 'bsi',
       'awei', 'tc_wetness', 'y', 'x', 'spatial_ref', 'time', 'dem'],
      dtype='object')

In [36]:
# Join the training data with the extracted values and remove unnecessary columns
training_array = pd.concat([training["LULC_code"], training_values], axis=1)

# Drop rows where there was no data available
training_array = training_array.dropna()

# Preview our resulting training array
training_array.head()

,LULC_code,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,nbi,ndmi,bsi,awei,tc_wetness,y,x,spatial_ref,time,dem
34,4,0.3070,0.1063,0.0852,0.1147,0.079907,1.329660e-07,0.000005,0.3332,0.3604,...,-0.125986,0.125986,-0.162418,-0.401925,-0.066308,-2393675.0,3876145.0,3832,2021-04-22,3.134878
35,2,0.3419,0.0842,0.0633,0.0989,0.156016,6.550729e-07,0.000010,0.3326,0.3506,...,-0.234519,0.234519,-0.320515,-0.489225,-0.047389,-2404505.0,3875165.0,3832,2021-04-22,27.563124
36,2,0.2105,0.0710,0.0717,0.0841,0.170746,1.638007e-06,0.000012,0.2695,0.3155,...,-0.053027,0.053027,-0.216642,-0.290075,-0.031707,-2404585.0,3875195.0,3832,2021-04-22,26.121586
37,2,0.3322,0.1085,0.0794,0.1169,0.133435,6.234050e-07,0.000008,0.3098,0.1713,...,-0.115327,0.115327,-0.198951,-0.480275,-0.067273,-2404405.0,3875085.0,3832,2021-04-22,29.110172
38,2,0.2817,0.1015,0.0749,0.1016,0.130518,6.958604e-07,0.000008,0.2729,0.2862,...,-0.062017,0.062017,-0.149952,-0.426325,-0.072172,-2404345.0,3875065.0,3832,2021-04-22,29.102854


In [37]:
print(training_array.shape[1], 'total columns')
print('columns included', training_array.columns)

38 total columns
columns included Index(['LULC_code', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'swir22_swir16', 'mci', 'ndci', 'nbi', 'ndmi', 'bsi',
       'awei', 'tc_wetness', 'y', 'x', 'spatial_ref', 'time', 'dem'],
      dtype='object')


In [38]:
print(training_array['LULC_code'].value_counts())
print('total gps points',(len(training_array)))

LULC_code
2    310
1    197
4    138
3    110
5    102
6     25
Name: count, dtype: int64
total gps points 882


In [39]:
# standard_schema = ['LULC_code', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
#        'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1',
#        'rededge2', 'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi',
#        'ndwi', 'mci', 'ndci', 'ndbi', 'y', 'x', 'time',
#        'spatial_ref']

In [40]:
# training_array=training_array[standard_schema]

In [41]:
training_array=training_array.drop(columns=["spatial_ref", "time"])

In [42]:
# Write the training data to a CSV file
training_array.to_csv(f"training-data/{version}-training.csv", index=False)

In [43]:
training_array["LULC_code"].dtype

dtype('int32')